# ATE, ATT, ATU condicionais, e os *MetaLearners*

Prof. Daniel de Abreu Pereira Uhr


### Estrutura da aula

* ATE, ATT, ATU condicionais
* MetaLearners 
  * SLearner, DRLearner, XLearner
  * CATE, CATT e CATU
  * MetaLearners com Variáveis Instrumentais

### Referências

* Cunningham, S. W. (2013). Causal inference: The mixtape. https://www.scunning.com/mixtape.html
* Facure, Matheus. Causal Inference for The Brave and True. https://matheusfacure.github.io/python-causality-handbook/landing-page.html 
* Joshua D. Angrist and Jörn-Steffen Pischke (2009). Mostly Harmless Econometrics: An Empiricist's Companion. Princeton University Press.
* Imbens, G. W., & Rubin, D. B. (2015). Causal inference in statistics, social, and biomedical sciences. Cambridge University Press.
* Itau Social (2018). Avaliação Econômica de Projetos Sociais. https://www.itausocial.org.br/wp-content/uploads/2018/05/avaliacao-economica-3a-ed_1513188151.pdf 
* Notas próprias


**Observações:** O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer *feedback* ou sugestão de melhoria.

---

## ATE, ATT e ATU Condicionais

### Conditional Average Treatment Effect (CATE)

O **CATE** é a extensão natural do ATE para ***subpopulações específicas***, definidas pelas covariáveis observáveis $X$.  
Formalmente:

$$
\beta^{CATE}(x) = \mathbb{E}[Y_i(1) - Y_i(0) \mid X_i = x].
$$

Ou, equivalentemente:

$$
\beta^{CATE}(x) = \mathbb{E}[\tau_i \mid X_i = x],
$$

onde $\tau_i = Y_i(1) - Y_i(0)$ é o efeito causal individual (ITE - *Individual Treatment Effect*). O CATE captura a **heterogeneidade de tratamento**, lembre que diferentes indivíduos podem apresentar diferentes ganhos ou perdas, dependendo de suas características.


### Conditional ATT (CATT)

O **CATT** restringe o efeito médio condicional à subpopulação dos tratados:

$$
\beta^{CATT}(x) = \mathbb{E}[Y_i(1) - Y_i(0) \mid X_i = x, D_i = 1].
$$

Interpretação: *entre os indivíduos com características $X=x$ que receberam o tratamento, qual foi o efeito médio observado?*



### Conditional ATU (CATU)

Analogamente, podemos definir o efeito médio condicional para os **não tratados**:

$$
\beta^{CATU}(x) = \mathbb{E}[Y_i(1) - Y_i(0) \mid X_i = x, D_i = 0].
$$

Interpretação: *entre os indivíduos com características $X=x$ que não receberam o tratamento, qual teria sido o efeito médio caso fossem tratados?*


### Por que Condicionar em $X$?

- **Relevância prática**: Políticas públicas raramente impactam a população de forma homogênea. O condicionamento permite identificar *quem mais se beneficia* e *quem menos se beneficia* do tratamento.  
- **Identificação de heterogeneidade**: Ao estimar o CATE, abrimos caminho para estudar efeitos diferenciais (por idade, renda, região, escolaridade, etc.).  
- **Construção de políticas direcionadas**: O CATT pode informar a continuidade do programa nos grupos atendidos, enquanto o CATU pode prever o impacto de expandir a intervenção a grupos ainda não tratados.  


### Hipóteses de Identificação do CATE

Para identificar o CATE a partir dos dados observados, precisamos assumir condições similares às do ATE, porém aplicadas de forma condicional:

1. **Ignorabilidade Condicional (ou Unconfoundedness):**

$$
(Y_i(0), Y_i(1)) \perp D_i \mid X_i
$$

Os resultados potenciais são independentes da designação ao tratamento, condicionalmente a $X_i$.

2. **Positividade (Overlap):**

$$
0 < P(D_i = 1 \mid X_i = x) < 1, \quad \forall x
$$

Todo subconjunto definido por $X$ deve ter probabilidade positiva de conter tratados e não tratados.

3. **Consistência:**

$$
Y_i = Y_i(D_i)
$$

O resultado observado corresponde ao resultado potencial associado ao status de tratamento efetivamente recebido.


### Relações com o ATE

O **ATE** pode ser expresso como média ponderada dos CATEs:

$$
ATE = \mathbb{E}[\beta^{CATE}(X)].
$$

De modo análogo:

$$
ATT = \mathbb{E}[\beta^{CATT}(X) \mid D=1],
\qquad
ATU = \mathbb{E}[\beta^{CATU}(X) \mid D=0].
$$

Ou seja, os efeitos médios populacionais (ATE, ATT, ATU) são agregações dos efeitos condicionais.



### Exemplo ilustrativo em python

Vamos criar um dataframe com 20 unidades, onde 10 unidades são tratadas ($D=1$) e 10 não tratadas ($D=0$). E esses individuos possuem covariáveis X, onde para alguns elas se repentem. E eles apresentam como resultado, $Y$.

| n | Y | D | X |
|---|---|---|---|
| 1 | 12 | 1 | 6 |
| 2 | 13 | 1 | 6 |
| 3 | 14 | 1 | 7 |
| 4 | 9 | 1 | 9 |
| 5 | 7 | 1 | 9 |
| 6 | 2 | 0 | 2 |
| 7 | 1 | 0 | 2 |
| 8 | 2 | 0 | 3 |
| 9 | 2 | 0 | 4 |
| 10| 5 | 0 | 4 |

Vamos aplicar o Outcome regression bilateral, e calcular o CATE.

In [41]:
import pandas as pd

# Criando o DataFrame a partir de um dicionário
data = {
    "Y": [12, 13, 14, 9, 7, 2, 1, 3, 4, 5],
    "D": [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    "X": [6, 6, 7, 9, 10, 2, 7, 3, 6, 6]
}
df = pd.DataFrame(data)

In [42]:
df

,Y,D,X
0,12,1,6
1,13,1,6
2,14,1,7
3,9,1,9
4,7,1,10
5,2,0,2
6,1,0,7
7,3,0,3
8,4,0,6
9,5,0,6


In [43]:
import statsmodels.formula.api as smf

reg1 = smf.ols('Y ~ X', data=df[df['D'] == 1]).fit()
df['mu_1']= reg1.predict(df)

reg0 = smf.ols('Y ~ X', data=df[df['D'] == 0]).fit()
df['mu_0']= reg0.predict(df)

df['ITE'] = df['mu_1'] - df['mu_0']

In [44]:
df

,Y,D,X,mu_1,mu_0,ITE
0,12,1,6,13.303030,3.127660,10.175371
1,13,1,6,13.303030,3.127660,10.175371
2,14,1,7,11.863636,3.234043,8.629594
3,9,1,9,8.984848,3.446809,5.538040
4,7,1,10,7.545455,3.553191,3.992263
5,2,0,2,19.060606,2.702128,16.358478
6,1,0,7,11.863636,3.234043,8.629594
7,3,0,3,17.621212,2.808511,14.812701
8,4,0,6,13.303030,3.127660,10.175371
9,5,0,6,13.303030,3.127660,10.175371


O CATE é média dos efeitos para aqueles indivíduos que têm a mesma condição, por exemplo $X=6$.

seria fazer a média entre os 4 indivíduos com $X=6$:
$$ \frac{(10.175 + 10.175 + 10.175 + 10.175)}{4} = 10.175 $$

In [45]:
# CATE para x = 6
df[df['X'] == 6]['ITE'].mean()

10.175370728562218

Para estimar o CATT e o CATU, basta fazer a média condicional no grupo dos tratados e não tratados, respectivamente.

In [ ]:
# CATT para x = 6 (Causal Average Treatment Effect on the Treated) avaliamos no grupo dos tratados
df[(df['X'] == 6) & (df['D'] == 1)]['ITE'].mean()

10.175370728562218

In [36]:
# CATU para x = 6 (Causal Average Treatment Effect on the Untreated) avaliamos no grupo dos não tratados
df[(df['X'] == 6) & (df['D'] == 0)]['ITE'].mean()

10.175370728562218

### CATE e os Meta-Learners

O termo **Meta-Learner** surgiu na interface entre **estatística, aprendizado de máquina e econometria aplicada**. A ideia central é tratar o problema de estimar efeitos causais heterogêneos como um **problema de aprendizado supervisionado em duas camadas** (*meta* = sobre algo).  

A intuição é simples: se pudermos aprender bem as funções de resposta ao tratamento e ao controle,

$$
\mu_1(x) = \mathbb{E}[Y \mid D=1, X=x], \qquad \mu_0(x) = \mathbb{E}[Y \mid D=0, X=x],
$$

então podemos estimar o efeito condicional como:

$$
\hat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x).
$$


- **Künzel, Sekhon, Bickel & Yu (2019)**. *Metalearners for estimating heterogeneous treatment effects using machine learning*.  
  - Propuseram uma taxonomia clara de *T-Learner, S-Learner e X-Learner*.  
  - Mostraram como diferentes algoritmos de ML podem ser usados como “base learners” dentro dessas arquiteturas.  
  - Este artigo é considerado o marco fundamental do campo.


#### Meta-Learners

Os *Meta-Learners* não são algoritmos em si, mas **estruturas de aprendizado**:  
- Cada *Meta-Learner* fornece uma *estratégia* para decompor o problema causal em subproblemas de predição.  
- A flexibilidade vem do fato de que qualquer algoritmo de machine learning pode ser usado como **base learner** (regressão linear, random forests, boosting, redes neurais etc.).  

Veremos:
* S-Learner
* T-Learner
* X-Learner
* DR-Learner

#### S-Learner

Recebe seu nome de "*Single Learner*" por estimar um único modelo para ambos os grupos.

- **Ideia**: Usar um único modelo para estimar $\mathbb{E}[Y \mid X, D]$, tratando $D$ como covariável.  
- **Etapas**:
  1. Ajustar um modelo $\hat{\mu}(x,d)$ sobre toda a amostra.
  2. Estimar o CATE como $\hat{\tau}(x) = \hat{\mu}(x,1) - \hat{\mu}(x,0)$.  

- **Força**: Usa toda a amostra de forma conjunta.  
- **Limitação**: Pode subestimar heterogeneidade se o modelo não interagir adequadamente $X$ e $D$.


#### T-Learner

Recebe seu nome de "*Two Learner*" por estimar dois modelos separados para os grupos tratado e não tratado.

- **Ideia**: Estimar dois modelos separados, um para tratados e outro para não tratados.  
- **Etapas**:
  1. Ajustar $\hat{\mu}_1(x)$ usando apenas a amostra dos tratados.
  2. Ajustar $\hat{\mu}_0(x)$ usando apenas a amostra dos não tratados.
  3. Estimar o CATE como $\hat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x)$.  

- **Força**: Flexibilidade em cada grupo.  
- **Limitação**: Pode ter alta variância se os grupos forem desbalanceados.


#### X-Learner

Recebe seu nome de "*Cross Learner*" por combinar informações de ambos os grupos.

- **Ideia**: Combinar o melhor do T e do S-Learner, útil especialmente quando há **desbalanceamento** entre grupos.  
- **Etapas**:
1. Estimar funções de resultado (como no T-Learner): $\hat{\mu}_1(x)$ e $\hat{\mu}_0(x)$.
2. Construir pseudo-efeitos:
   1. Para cada tratado, pergunto: “O quanto ele se saiu melhor do que se fosse controle?”
      1. $\hat{\tau}_i^T = Y_i - \hat{\mu}_0(X_i)$.
   2. Para cada controle, pergunto: “O quanto ele se saiu pior do que se fosse tratado?”
      1. $\hat{\tau}_i^C = \hat{\mu}_1(X_i) - Y_i$.
3. Modelar os pseudo-efeitos separadamente:
   1. Estimo $\tau_T(x)$ usando os pseudo-efeitos dos tratados.
   2. Estimo $\tau_C(x)$ usando os pseudo-efeitos dos controles.
   3. Combinar os dois modelos com pesos baseados no propensity score $p(X)$:
      1. Se $p(X)$ for alto (indivíduo tem mais chance de ser tratado), confio mais em $\tau_T(x)$.
      2. Se $p(X)$ for baixo (mais chance de ser controle), confio mais em $\tau_C(x)$.
   4. Assim, o estimador final é uma média ponderada:
   5. $ \hat{\tau}(x) = g(x) \hat{\tau}_T(x) + (1-g(x)) \hat{\tau}_C(x)$
      1. onde $g(x)$ geralmente é escolhido como $p(X)$.

- **Força**: Melhor desempenho em dados desbalanceados.  
- **Limitação**: Requer duas camadas de modelagem.

#### DR-Learner

Recebe seu nome de "*Doubly Robust Learner*" por combinar dois modelos.

- **Ideia**: Baseado na fórmula **duplamente robusta** (AIPW), corrige erros de especificação de modelos de resposta ou de propensão.  
- **Etapas**:
  1. Estimar $\hat{\mu}_1(x)$, $\hat{\mu}_0(x)$ e $\hat{p}(x)$.
  2. Construir pseudo-outcomes:

     $$
     \tilde{Y}_i = \hat{\mu}_1(X_i) - \hat{\mu}_0(X_i) \;+\; \frac{D_i}{\hat{p}(X_i)}(Y_i - \hat{\mu}_1(X_i)) - \frac{1-D_i}{1-\hat{p}(X_i)}(Y_i - \hat{\mu}_0(X_i)).
     $$

  3. Regressar $\tilde{Y}_i$ sobre $X_i$ para estimar $\tau(x)$.  

- **Força**: Consistência se pelo menos um dos modelos estiver correto.  
- **Limitação**: Computacionalmente mais exigente.

#### Síntese

- Todos os *Meta-Learners* têm como objetivo **recuperar o CATE**.  
- O **ATE, ATT e ATU** podem ser obtidos como médias ponderadas dos CATEs:  

$$
ATE = \mathbb{E}[\tau(X)], \quad
ATT = \mathbb{E}[\tau(X) \mid D=1], \quad
ATU = \mathbb{E}[\tau(X) \mid D=0].
$$


### Aplicação em Python

In [46]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [48]:
# DataFrame "data"
data = pd.read_stata("https://github.com/Daniel-Uhr/data/raw/main/cattaneo2.dta")

In [49]:
data

,bweight,mmarried,mhisp,fhisp,foreign,alcohol,deadkids,mage,medu,fage,...,order,msmoke,mbsmoke,mrace,frace,prenatal,birthmonth,lbweight,fbaby,prenatal1
0,3459,married,0,0,0,0,0,24,14,28,...,2,0 daily,nonsmoker,1,1,1,12,0,No,Yes
1,3260,notmarried,0,0,1,0,0,20,10,0,...,3,0 daily,nonsmoker,0,0,1,7,0,No,Yes
2,3572,married,0,0,1,0,0,22,9,30,...,3,0 daily,nonsmoker,1,1,1,3,0,No,Yes
3,2948,married,0,0,0,0,0,26,12,30,...,2,0 daily,nonsmoker,1,1,1,1,0,No,Yes
4,2410,married,0,0,0,0,0,20,12,21,...,1,0 daily,nonsmoker,1,1,1,3,1,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4637,3317,notmarried,0,0,0,0,0,21,12,24,...,1,1-5 daily,smoker,1,1,2,10,0,Yes,No
4638,3030,married,0,0,0,1,1,30,12,23,...,2,11+ daily,smoker,1,1,2,2,0,No,No
4639,2950,notmarried,0,0,0,0,0,23,12,0,...,1,0 daily,nonsmoker,0,0,1,5,0,Yes,Yes
4640,3969,married,0,0,0,0,0,23,12,25,...,1,0 daily,nonsmoker,1,1,1,2,0,Yes,Yes


In [50]:
#Filtragem dos dados

# Criar a variável de resultado
data['Y'] = data['bweight']

# Crie a variável 'D' ("Treated" ou tratado) com valor inicial de 0
data['D'] = 0
# Recodifique 'D' para 1 se 'mbsmoke' for igual a 'smoker'
data.loc[data['mbsmoke'] == 'smoker', 'D'] = 1

# Criar variáveis de controle
data['Mmarried'] = 0
data.loc[data['mmarried'] == 'married', 'Mmarried'] = 1
data['casada'] = 0
data.loc[data['mmarried']=='married', 'casada'] = 1

Vamos utilizar os MetaLearners do EconML.

* `SLearner`
* `TLearner`
* `XLearner`
* `DRLearner`

In [60]:
from econml.metalearners import SLearner, TLearner, XLearner
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore", message="'force_all_finite'")


In [54]:
y = data['Y']
t = data['D']
X = data[['casada', 'mage', 'medu', 'fage', 'fedu', 'nprenatal']]

In [56]:
# exemplo com T-Learner
t_learner = TLearner(models=[LinearRegression(),LinearRegression()])

t_learner.fit(Y=y, T=t, X=X)

# CATE estimado
t_learner.effect(X)


'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


array([-181.15967751, -105.72287911, -190.16800053, ..., -175.8348929 ,
       -202.85682542, -188.9998092 ])

In [ ]:
# Exemplo: CATE para uma mãe de 25 anos, ensino médio 12 anos, casada,
# pai com 12 anos de escolaridade e 8 consultas pré-natal
X_example = pd.DataFrame({
    'casada': [1],
    'mage': [25],
    'medu': [12],
    'fage': [27],
    'fedu': [12],
    'nprenatal': [8]
})

cate_1 = t_learner.effect(X_example)

In [ ]:
print(cate_1.

[-192.49812068]


In [58]:
cate_hat = t_learner.effect(X)

# ATE: média dos efeitos em toda a população
ATE_hat = np.mean(cate_hat)

# ATT: média dos efeitos apenas nos tratados (mães fumantes)
ATT_hat = np.mean(cate_hat[t == 1])

# ATU: média dos efeitos apenas nos não tratados (mães não fumantes)
ATU_hat = np.mean(cate_hat[t == 0])

print("ATE estimado:", ATE_hat)
print("ATT estimado:", ATT_hat)
print("ATU estimado:", ATU_hat)

ATE estimado: -227.12758124162156
ATT estimado: -201.8403874939326
ATU estimado: -232.91057102404702


'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.


In [ ]:
# Rodando o SLearner
# S-Learner: one model E[Y|X, D]
s_learner = SLearner(overall_model=mu_model_all, discrete_treatment=True)
s_learner.fit(Y=y_train, T=t_train, X=X_train)

In [ ]:
# T-Learner: two models E[Y|X, D=1] and E[Y|X, D=0]
t_learner = TLearner(models=[mu_model_0, mu_model_1], discrete_treatment=True)
t_learner.fit(Y=y_train, T=t_train, X=X_train)

In [ ]:
# X-Learner: combines pseudo-effects; provide outcome models and propensity (optional but recommended)
x_learner = XLearner(
    outcome_models=[mu_model_0, mu_model_1],
    effect_models=[tau_model_control, tau_model_treated],
    propensity_model=prop_model,
    discrete_treatment=True
)
x_learner.fit(Y=y_train, T=t_train, X=X_train)

In [ ]:
# DR-Learner: doubly robust (AIPW-based)
dr_learner = DRLearner(
    model_propensity=prop_model,
    model_regression=make_pipeline(preprocess, base_reg),  # fits E[Y|X,D] internally
    model_final=final_tau_model,
    discrete_treatment=True
)
dr_learner.fit(Y=y_train, T=t_train, X=X_train)